## Project 2: Recommendation Engines

In [2]:
import pandas as pd

In [3]:
import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline

In [44]:
from numpy import nan

In [48]:
from sklearn.metrics.pairwise import cosine_similarity as cos 

In [811]:
from sklearn.metrics import mean_squared_error as sk_rmse

In [86]:
import numpy as np

### 1. The dataset

The movie-lens dataset 

In [4]:
path="C:/Users/talha/Documents/Training/CUNY Classes/Data612/Week1/ml-latest-small/"

In [5]:
ratings=pd.read_csv(path+'ratings.csv')

In [622]:
movie_names=pd.read_csv(path+'movies.csv')

In [6]:
#ratings by user and movie
ratings.head(n=5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
#simple summary statistics
ratings.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [623]:
movie_names.head(n=10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [26]:
#calculate mean rating by user
user_mean=ratings[['userId','rating']].groupby('userId').mean().rename(columns={'rating':'user_avg'})

In [29]:
user_mean.head(n=5)

,user_avg
userId,
1,4.366379
2,3.948276
3,2.435897
4,3.555556
5,3.636364


In [27]:
#calculate mean rating by movie
movie_mean=ratings[['movieId','rating']].groupby('movieId').mean().rename(columns={'rating':'movie_avg'})

In [28]:
movie_mean.head(n=5)

,movie_avg
movieId,
1,3.920930
2,3.431818
3,3.259615
4,2.357143
5,3.071429


In [32]:
ratings=ratings.merge(user_mean, left_on='userId',right_on='userId')

In [34]:
ratings=ratings.merge(movie_mean, left_on='movieId',right_on='movieId')

In [35]:
ratings.head(n=5)

,userId,movieId,rating,timestamp,user_avg,movie_avg
0,1,1,4.0,964982703,4.366379,3.92093
1,5,1,4.0,847434962,3.636364,3.92093
2,7,1,4.5,1106635946,3.230263,3.92093
3,15,1,2.5,1510577970,3.448148,3.92093
4,17,1,4.5,1305696483,4.209524,3.92093


In [36]:
#standardize the ratings
ratings['user_std']=ratings['rating']-ratings['user_avg']
ratings['movie_std']=ratings['rating']-ratings['movie_avg']

In [37]:
ratings.head(n=5)

,userId,movieId,rating,timestamp,user_avg,movie_avg,user_std,movie_std
0,1,1,4.0,964982703,4.366379,3.92093,-0.366379,0.07907
1,5,1,4.0,847434962,3.636364,3.92093,0.363636,0.07907
2,7,1,4.5,1106635946,3.230263,3.92093,1.269737,0.57907
3,15,1,2.5,1510577970,3.448148,3.92093,-0.948148,-1.42093
4,17,1,4.5,1305696483,4.209524,3.92093,0.290476,0.57907


In [657]:
ratings[ratings['userId']==3][0:10]

,userId,movieId,rating,timestamp,user_avg,movie_avg,user_std,movie_std
3571,3,527,0.5,1306464275,2.435897,4.225000,-1.935897,-3.725000
8897,3,1275,3.5,1306463323,2.435897,3.729167,1.064103,-0.229167
9859,3,1587,4.5,1306464003,2.435897,3.258065,2.064103,1.241935
10741,3,2018,0.5,1306464175,2.435897,3.368421,-1.935897,-2.868421
11158,3,2090,0.5,1306464261,2.435897,3.192308,-1.935897,-2.692308
11244,3,2105,2.0,1306463559,2.435897,3.340000,-0.435897,-1.340000
15986,3,3703,5.0,1306463603,2.435897,4.037500,2.564103,0.962500
18092,3,31,0.5,1306463578,2.435897,3.184211,-1.935897,-2.684211
18130,3,647,0.5,1306463619,2.435897,3.327586,-1.935897,-2.827586
18159,3,688,0.5,1306464228,2.435897,2.333333,-1.935897,-1.833333


In [619]:
#identify the most rated movies 
rating_count=ratings[['movieId','rating']].groupby('movieId').count().rename(columns={'rating':'rating_count'})

In [660]:
rating_count=rating_count.sort_values(['rating_count'],ascending=False)

In [661]:
rating_count.head(n=5)

,rating_count
movieId,
356,329
318,317
296,307
593,279
2571,278


In [625]:
#identify the most profilic raters
movies_rated=ratings[['userId','rating']].groupby('userId').count().rename(columns={'rating':'movie_count'})

In [647]:
mvr_rat_s=movies_rated.sort_values(['movie_count'],ascending=False)

In [656]:
mvr_rat_s.head(n=5)

,movie_count
userId,
414,2698
599,2478
474,2108
448,1864
274,1346


In [659]:
user_top=list(mvr_rat_s.index[0:20])

In [662]:
movie_top=list(rating_count.index[0:20])

### 2. Collaborative Filtering

#### 2.1 User - User Collaborative Filtering

In [38]:
#reshape the data
rt_w_user=ratings[['userId','movieId','user_std']].pivot(index='userId', columns='movieId',values='user_std')

In [39]:
rt_w_user.head(n=5)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.366379,NaN,-0.366379,NaN,NaN,-0.366379,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.363636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
rt_w_user_i=rt_w_user.replace(nan,0)

In [49]:
a=rt_w_user_i.head(n=5) 

In [217]:
b=cos(a,a)

In [435]:
b=rt_w_user.head(n=5)

In [70]:
len(a.columns)

9724

In [71]:
len(a)

5

In [153]:
z=[la for _,la in sorted(zip(lb,la),reverse=True)]

In [221]:
z=[la for lb,la in sorted(zip(lb,la),reverse=True)]

In [106]:
for az, bz in zip(list(a.index),list(b[1])):
    print(az,bz)

1 0.0012645157377626504
2 0.9999999999999999
3 0.0
4 -0.017164020897390282
5 0.021795713694264633


In [150]:
weight_k(lb,la,4)

1.96383239545234

In [275]:
v=cos(a,a)

In [294]:
v[1][0:2]

array([0.00126452, 1.        ])

In [300]:
b=list(a.index)

In [302]:
b[0:3]

[1, 2]

In [304]:
v[1][0:3]

array([0.00126452, 1.        , 0.        ])

In [257]:
vb=list(v[2])

In [262]:
z=[(b,vb) for vb,b in sorted(zip(vb,b),reverse=True)]

In [274]:
sorted(vb,reverse=True)

[0.9999999999999997,
 0.0005525771757217378,
 0.0,
 -0.011259777588577888,
 -0.031538923337686174]

In [209]:
a

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.366379,0.0,-0.366379,0.0,0.0,-0.366379,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.363636,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:
#define a weighting average function for k users
#rj is a set of ratings for item j
#sa are the similarities for users / items. 
def weight_k(sa,rj,k):
    return(np.dot(sa[0:k],rj[0:k])/sum(sa[0:k]))

In [423]:
#define function to find the Neighborhood of user / item
#find the similarity of each user / item, and identify top k users / items
#return the neighborhood of users / items
def Neighbor_sim(a,k): # a is a matrix of user and items, and neighborhood of k
    items=np.arange(len(a))
    sim=cos(a,a)
    ia=list(a.index)
    Nlist=pd.DataFrame()
    Slist=pd.DataFrame()
    k=k+1 #add one since the first includes the user himself
    for i in items:
        ib=list(sim[i])
        z=[ia for ib,ia in sorted(zip(ib,ia),reverse=True)]
        Slist=Slist.append(pd.DataFrame(sorted(sim[i],reverse=True)[0:k]).transpose())
        Nlist=Nlist.append(pd.DataFrame(np.array(z[0:k]).reshape(1,k)))
    Slist['index']=ia
    Slist=Slist.set_index('index')
    Slist.drop([0],axis=1,inplace=True)
    Nlist=Nlist.set_index(0)
    return (Nlist,Slist)

In [382]:
c,d=Neighbor_sim(a,3) 

In [624]:
#next step is to filter out the movies of interest
#then use the ratings provided by users (based on those users who are most similar)
#ratings are to be weighted by similarities of those users who are most similar. 

In [392]:
list(c.iloc[0,:])

[4, 5]

In [741]:
#for user i, and given a set of movies, predict highest rated / recommended movies for user
def user_collab(usr,it,usr_mat,k):
#user is a list of users for whom we want to predict ratings 
#it is a list of items / movies
#user_mat is a dense matrix, as a dataframe, of users (as rows) and items (across columns)
#k is a parameter for the size of the neighborhood 
    usr=list(usr)
    it=list(it)
    usrid,sims=Neighbor_sim(usr_mat,k) #calculate similarities and neighborhoods of all users
    predict=[]
    for e in usr: # for each user, need to exclude select user and focus on similar user
        #filt_list=[all_usr for all_usr in all_usr if all_usr!=e] #filter the list so that current user is excluded
        for i in it: #for each movie / item
            usr_mat_s=usr_mat.filter(items=list([i]),axis=1) #filter for movies of interest
            clist_u=usrid.loc[e,:] #return the most similar users for user e
            usr_mat_s_r=usr_mat_s.loc[clist_u] #filter matrix, i.e. row filter
            usr_mat_s_r=usr_mat_s_r.squeeze()
            sims_s=sims.loc[e,:]
            rt=weight_k(sims_s,usr_mat_s_r,k)
            predict.append([e,i,rt])
            #need to calculate the weights here
    predict=pd.DataFrame(predict,columns=['userid','item','rating'])
    return (predict)

In [742]:
#run a test on the data
users=list([1,2,3,4,5])
item=list([1,2,3])
user_collab(users,item,a,3)

,userid,item,rating
0,1,1,0.111062
1,1,2,0.000000
2,1,3,0.000000
3,2,1,0.323606
4,2,2,0.000000
5,2,3,-0.020091
6,3,1,0.018908
7,3,2,-0.000000
8,3,3,0.018908
9,4,1,-0.887209


In [669]:
#run an analysis on top 20 movies and top 20 most profilic raters
#movie_top - top movies
#user_top - most prolific raters
top_ratings=rt_w_user_i.filter(items=movie_top,axis=1) # missing values have been imputed zero

In [670]:
top_ratings=top_ratings.filter(items=user_top,axis=0)

In [673]:
top_ratings.tail(n=5)

movieId,356,318,296,593,2571,260,480,110,589,527,2959,1,1196,50,2858,47,780,150,1198,4993
userId,,,,,,,,,,,,,,,,,,,,
298,0.636315,1.136315,2.136315,0.000000,1.636315,1.636315,1.636315,0.636315,1.636315,0.000000,2.636315,-0.363685,1.636315,1.136315,1.636315,1.636315,0.136315,0.000000,2.136315,1.636315
177,1.624447,1.624447,1.624447,1.624447,-0.375553,-0.875553,0.124447,0.000000,0.000000,1.124447,0.624447,1.624447,-0.375553,0.624447,0.624447,1.124447,0.124447,0.624447,0.124447,-1.375553
318,0.744027,0.244027,0.744027,0.244027,-0.255973,-0.755973,0.244027,0.000000,0.000000,0.244027,0.000000,0.000000,-0.755973,0.000000,-0.255973,0.244027,0.244027,0.000000,0.000000,0.244027
232,0.749420,1.749420,1.749420,0.000000,1.249420,1.249420,-0.250580,1.249420,0.749420,1.249420,1.249420,0.249420,1.249420,0.000000,0.000000,1.249420,0.249420,1.249420,1.249420,0.249420
480,1.724880,1.724880,0.724880,1.224880,1.724880,1.224880,1.724880,1.724880,1.224880,1.724880,1.224880,-0.275120,1.724880,0.224880,0.724880,1.224880,-0.275120,0.224880,1.724880,0.224880


We note that there a number of missing ratings for userid 318. 

In [743]:
top_pred=user_collab(user_top,movie_top,top_ratings,5)

In [744]:
top_pred.head(n=5)

,userid,item,rating
0,414,356,1.050334
1,414,318,1.259108
2,414,296,1.484199
3,414,593,0.522189
4,414,2571,1.571071


In [745]:
top_pred[top_pred['userid']==318]

,userid,item,rating
340,318,356,1.510847
341,318,318,1.525644
342,318,296,1.685563
343,318,593,1.448773
344,318,2571,0.367186
345,318,260,-0.048107
346,318,480,0.145510
347,318,110,0.526197
348,318,589,-0.010568
349,318,527,1.169454


In [726]:
si.loc[ui.loc[414,:]]

,1,2,3,4,5
index,,,,,
599,0.940870,0.913683,0.821308,0.808172,0.780866
298,0.940870,0.893728,0.817416,0.791318,0.773640
274,0.849160,0.808172,0.800488,0.782631,0.768255
480,0.843250,0.816582,0.815498,0.791318,0.782631
249,0.828809,0.821308,0.817416,0.816582,0.723039


In [458]:
a.loc[a.index.isin(list(one.iloc[0,:]))]

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.363636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [380]:
elist=list(a.index)
d=[elist for elist in elist if elist !=1]
print(d)

[2, 3, 4, 5]


In [378]:
a.filter(items=c,axis=1).loc[~a.index.isin(c)]

movieId,1,4,5
userId,,,
2,0.0,0.0,0.0
3,0.0,0.0,0.0


#### 2.2 Item - Item Collaborative Filtering

In [749]:
#reshape the data
rt_w_mov=ratings[['userId','movieId','movie_std']].pivot(index='movieId', columns='userId',values='movie_std')

In [750]:
rt_w_mov.head(n=5)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.079070,NaN,NaN,NaN,0.07907,NaN,0.57907,NaN,NaN,NaN,...,0.07907,NaN,0.07907,-0.920930,0.079070,-1.42093,0.07907,-1.420930,-0.92093,1.07907
2,NaN,NaN,NaN,NaN,NaN,0.568182,NaN,0.568182,NaN,NaN,...,NaN,0.568182,NaN,1.568182,0.068182,NaN,NaN,-1.431818,NaN,NaN
3,0.740385,NaN,NaN,NaN,NaN,1.740385,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.259615,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,0.642857,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,1.928571,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-0.071429,NaN,NaN,NaN,NaN,NaN,NaN


In [751]:
rt_w_mov_i=rt_w_mov.replace(nan,0)

In [752]:
b=rt_w_mov_i.head(n=5)

In [753]:
b

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.079070,0.0,0.0,0.0,0.07907,0.000000,0.57907,0.000000,0.0,0.0,...,0.07907,0.000000,0.07907,-0.920930,0.079070,-1.42093,0.07907,-1.420930,-0.92093,1.07907
2,0.000000,0.0,0.0,0.0,0.00000,0.568182,0.00000,0.568182,0.0,0.0,...,0.00000,0.568182,0.00000,1.568182,0.068182,0.00000,0.00000,-1.431818,0.00000,0.00000
3,0.740385,0.0,0.0,0.0,0.00000,1.740385,0.00000,0.000000,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,-1.259615,0.00000,0.00000
4,0.000000,0.0,0.0,0.0,0.00000,0.642857,0.00000,0.000000,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000
5,0.000000,0.0,0.0,0.0,0.00000,1.928571,0.00000,0.000000,0.0,0.0,...,0.00000,0.000000,0.00000,-0.071429,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000


In [758]:
top_ratings_mov=rt_w_mov_i.filter(items=movie_top,axis=0) # missing values have been imputed zero

In [759]:
top_rating_mov=top_ratings_mov.filter(items=user_top,axis=1)

In [760]:
top_rating_mov.head(n=5)

userId,414,599,474,448,274,610,68,380,606,288,249,387,182,307,603,298,177,318,232,480
movieId,,,,,,,,,,,,,,,,,,,,
356,0.835866,-0.664134,-1.164134,-1.164134,0.335866,-1.164134,-0.664134,0.835866,-0.164134,0.835866,0.335866,-0.164134,0.835866,-0.164134,-1.164134,-1.164134,0.835866,0.335866,-0.164134,0.835866
318,0.570978,-0.429022,0.570978,0.000000,0.070978,-1.429022,-1.429022,-1.429022,-0.929022,0.570978,0.070978,-0.929022,0.070978,0.070978,0.000000,-0.929022,0.570978,-0.429022,0.570978,0.570978
296,0.802932,0.802932,-0.197068,0.802932,0.802932,0.802932,-2.197068,0.802932,0.802932,0.802932,-0.197068,0.802932,0.802932,0.302932,0.802932,0.302932,0.802932,0.302932,0.802932,-0.197068
593,-0.161290,-1.161290,0.338710,0.838710,-0.161290,0.338710,-0.661290,0.838710,0.338710,0.838710,-0.161290,-0.161290,0.338710,0.338710,0.838710,0.000000,0.838710,-0.161290,0.000000,0.338710
2571,0.807554,0.807554,0.307554,-2.192446,-0.192446,0.807554,0.307554,0.307554,0.807554,-1.192446,0.807554,-0.192446,0.807554,-0.692446,0.807554,-0.192446,-1.192446,-0.692446,0.307554,0.807554


In [755]:
#for user i, and given a set of movies, predict highest rated / recommended movies for user
def item_collab(usr,it,item_mat,k):
#user is a list of users for whom we want to predict ratings 
#it is a list of items / movies
#user_mat is a dense matrix, as a dataframe, of users (as rows) and items (across columns)
#k is a parameter for the size of the neighborhood 
    usr=list(usr)
    it=list(it)
    itemid,sims=Neighbor_sim(item_mat,k) #calculate similarities and neighborhoods of all users
    predict=[]
    for i in it: # for each movie / item
        for e in usr: # for each user 
            item_mat_s=item_mat.filter(items=list([e]),axis=1) #filter for movies of interest
            clist_i=itemid.loc[i,:] #return the most similar movies for user e
            item_mat_s_r=item_mat_s.loc[clist_i] #filter matrix, i.e. row filter
            item_mat_s_r=item_mat_s_r.squeeze()
            sims_s=sims.loc[i,:]
            rt=weight_k(sims_s,item_mat_s_r,k)
            predict.append([e,i,rt])
            #need to calculate the weights here
    predict=pd.DataFrame(predict,columns=['userid','item','rating'])
    return (predict)

In [746]:
#do a simple comparison of user-user 
user_collab(users,item,a,3).head(n=5)

,userid,item,rating
0,1,1,0.111062
1,1,2,0.000000
2,1,3,0.000000
3,2,1,0.323606
4,2,2,0.000000


In [756]:
#and item to item
item_collab(users,item,b,3).head(n=5)

,userid,item,rating
0,1,1,0.255642
1,2,1,0.000000
2,3,1,0.000000
3,4,1,0.000000
4,5,1,0.000000


In [761]:
top_pred_item=item_collab(user_top,movie_top,top_ratings_mov,5)

In [762]:
top_pred_item.head(n=5)

,userid,item,rating
0,414,356,0.561114
1,599,356,-0.192496
2,474,356,0.064332
3,448,356,-0.339226
4,274,356,0.063426


In [764]:
top_pred[top_pred['item']==356]

,userid,item,rating
0,414,356,1.050334
20,599,356,1.194965
40,474,356,1.124954
60,448,356,0.857174
80,274,356,1.238752
100,610,356,0.775026
120,68,356,0.730544
140,380,356,0.470385
160,606,356,1.282409
180,288,356,1.120518


#### 2.3 Expanding Item - Item Filtering

### 3. Evaluating the Recommendations

#### 3.1 Developing the Baseline Predictor

In [765]:
ratings.head(n=5)

,userId,movieId,rating,timestamp,user_avg,movie_avg,user_std,movie_std
0,1,1,4.0,964982703,4.366379,3.92093,-0.366379,0.07907
1,5,1,4.0,847434962,3.636364,3.92093,0.363636,0.07907
2,7,1,4.5,1106635946,3.230263,3.92093,1.269737,0.57907
3,15,1,2.5,1510577970,3.448148,3.92093,-0.948148,-1.42093
4,17,1,4.5,1305696483,4.209524,3.92093,0.290476,0.57907


In [767]:
#calculate mean rating across all ratings
#user_mean=ratings[['userId','rating']].groupby('userId').mean().rename(columns={'rating':'user_avg'})
all_mean=ratings[['rating']].mean()

In [774]:
ratings['all_mean']=all_mean.values[0]

In [776]:
ratings['baseline_pred']=(ratings['movie_avg']-ratings['all_mean'])+(ratings['user_avg']-ratings['all_mean'])+ratings['all_mean']

In [778]:
ratings['baseline_diff']=ratings['rating']-ratings['baseline_pred']

In [779]:
ratings.head(n=5)

,userId,movieId,rating,timestamp,user_avg,movie_avg,user_std,movie_std,all_mean,baseline_pred,baseline_diff
0,1,1,4.0,964982703,4.366379,3.92093,-0.366379,0.07907,3.501557,4.785753,-0.785753
1,5,1,4.0,847434962,3.636364,3.92093,0.363636,0.07907,3.501557,4.055737,-0.055737
2,7,1,4.5,1106635946,3.230263,3.92093,1.269737,0.57907,3.501557,3.649636,0.850364
3,15,1,2.5,1510577970,3.448148,3.92093,-0.948148,-1.42093,3.501557,3.867521,-1.367521
4,17,1,4.5,1305696483,4.209524,3.92093,0.290476,0.57907,3.501557,4.628897,-0.128897


#### 3.2 Developing Predictions for Evaluation

In [785]:
rt_w_mov.head(n=5)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.079070,NaN,NaN,NaN,0.07907,NaN,0.57907,NaN,NaN,NaN,...,0.07907,NaN,0.07907,-0.920930,0.079070,-1.42093,0.07907,-1.420930,-0.92093,1.07907
2,NaN,NaN,NaN,NaN,NaN,0.568182,NaN,0.568182,NaN,NaN,...,NaN,0.568182,NaN,1.568182,0.068182,NaN,NaN,-1.431818,NaN,NaN
3,0.740385,NaN,NaN,NaN,NaN,1.740385,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.259615,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,0.642857,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,1.928571,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-0.071429,NaN,NaN,NaN,NaN,NaN,NaN


In [790]:
#pd.wide_to_long(rt_w_mov,stubnames='movie_std',i='movieId',j='userId')
#rt_w_mov.pivot(index='movieId', columns='userId',values='movie_std').head(n=5)
#rt_w_mov.melt(id_vars=['movieId'],var_name='userId',value_name='movie_std')
rt_w_mov['movieId']=rt_w_mov.index

In [792]:
rt_l_mov=rt_w_mov.melt(id_vars=['movieId'],var_name='userId',value_name='movie_std')

In [806]:
rt_l_mov[rt_l_mov.movie_std.isnull()].head(n=5)

,movieId,userId,movie_std
1,2,1,NaN
3,4,1,NaN
4,5,1,NaN
6,7,1,NaN
7,8,1,NaN


In [807]:
user_top100=list(mvr_rat_s.index[0:100])

In [808]:
movie_top100=list(rating_count.index[0:100])

In [758]:
top_ratings_mov=rt_w_mov_i.filter(items=movie_top100,axis=0) # missing values have been imputed zero

In [759]:
top_rating_mov=top_ratings_mov.filter(items=user_top100,axis=1) #item based dataset

In [809]:
top_ratings=rt_w_user_i.filter(items=movie_top100,axis=1) # missing values have been imputed zero

In [810]:
top_ratings=top_ratings.filter(items=user_top100,axis=0) #user based dataset

In [820]:
ratings.head()

,userId,movieId,rating,timestamp,user_avg,movie_avg,user_std,movie_std,all_mean,baseline_pred,baseline_diff
0,1,1,4.0,964982703,4.366379,3.92093,-0.366379,0.07907,3.501557,4.785753,-0.785753
1,5,1,4.0,847434962,3.636364,3.92093,0.363636,0.07907,3.501557,4.055737,-0.055737
2,7,1,4.5,1106635946,3.230263,3.92093,1.269737,0.57907,3.501557,3.649636,0.850364
3,15,1,2.5,1510577970,3.448148,3.92093,-0.948148,-1.42093,3.501557,3.867521,-1.367521
4,17,1,4.5,1305696483,4.209524,3.92093,0.290476,0.57907,3.501557,4.628897,-0.128897


In [824]:
sk_rmse(ratings.rating,ratings.baseline_pred)

0.6753181234090172